## 53단계: 모델 저장 및 읽어오기기

> .

### 53.1 넘파이의 save 함수와 load 함수

`np.save`, `np.load` 함수를 알아보자.

In [1]:
import numpy as np

x = np.array([1, 2, 3])
np.save('test.npy', x)

x = np.load('test.npy')
print(x)

[1 2 3]


In [2]:
x1 = np.array([1, 2, 3])
x2 = np.array([4, 5, 6])

np.savez('test.npz', x1=x1, x2=x2)

arrays = np.load('test.npz')
x1 = arrays['x1']
x2 = arrays['x2']
print(x1)
print(x2)

[1 2 3]
[4 5 6]


In [3]:
x1 = np.array([1, 2, 3])
x2 = np.array([4, 5, 6])
data = {'x1':x1, 'x2':x2}

np.savez('test.npz', **data)

arrays = np.load('test.npz')
x1 = arrays['x1']
x2 = arrays['x2']
print(x1)
print(x2)

[1 2 3]
[4 5 6]


In [4]:
x1 = np.array([1, 2, 3])
x2 = np.array([4, 5, 6])
data = {'x1':x1, 'x2':x2}

# 압축 후 저장
np.savez_compressed('test.npz', **data)

arrays = np.load('test.npz')
x1 = arrays['x1']
x2 = arrays['x2']
print(x1)
print(x2)

[1 2 3]
[4 5 6]


### 53.2 Layer 클래스의 매개변수를 평평하게

계층은 아래 그림과 같이 Layer가 중첩된 구조를 갖는다.

```python
layer = Layer()

l1 = Layer()
l1.p1 = Parameter(np.array(1))

layer.l1 = l1
layer.p2 = Parameter(np.array(2))
layer.p3 = Parameter(np.array(3))
```

<img src="images/그림 53-1.png" width=400/>

이때 Parameter를 '하나의 평평한 딕셔너리'로 뽑아보자.

In [5]:
# dezero/layers.py

class Layer:
    ...
    
    def _flatten_params(self, params_dict, parent_key=''):
        for name in self._params:
            obj = self.__dict__[name]
            key = parent_key + '/' + name if parent_key else name
            
            if isinstance(obj, Layer):
                obj.__flatten_params(params_dict, key)
            else:
                params_dict[key] = obj

In [6]:
from dezero import Layer
from dezero.core import Parameter

layer = Layer()

l1 = Layer()
l1.p1 = Parameter(np.array(1))

layer.l1 = l1
layer.p2 = Parameter(np.array(2))
layer.p3 = Parameter(np.array(3))

params_dict = {}
layer._flatten_params(params_dict)
print(params_dict)

{'l1/p1': Variable(1), 'p3': Variable(3), 'p2': Variable(2)}


### 53.3 Layer 클래스의 save 함수와 load 함수

In [7]:
# dezero/layers.py

import os

class Layer:
    ...
    
    def save_weights(self, path):
        self.to_cpu()

        params_dict = {}
        self._flatten_params(params_dict)
        array_dict = {key: param.data for key, param in params_dict.items()
                      if param is not None}
        try:
            np.savez_compressed(path, **array_dict)
        except (Exception, KeyboardInterrupt) as e:
            if os.path.exists(path):
                os.remove(path)
            raise

    def load_weights(self, path):
        npz = np.load(path)
        params_dict = {}
        self._flatten_params(params_dict)
        for key, param in params_dict.items():
            param.data = npz[key]

In [8]:
# MNIST 학습 예시

import os
import dezero
import dezero.dataloaders
import dezero.functions as F
from dezero import optimizers
from dezero import DataLoader
from dezero.models import MLP

max_epoch = 3
batch_size = 100

train_set = dezero.datasets.MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)
model = MLP((1000, 10))
optimizer = optimizers.SGD().setup(model)

for epoch in range(max_epoch):
    sum_loss = 0
    
    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(t)
        
    print('epoch: {}, loss: {:4f}'.format(
        epoch + 1, sum_loss / len(train_set)
    ))

# 테스트셋 점수 확인
test_set = dezero.datasets.MNIST(train=False)
test_loader = DataLoader(test_set, batch_size)
sum_acc = 0
with dezero.no_grad():
    for x, t in test_loader:
        y = model(x)
        acc = F.accuracy(y, t)
        sum_acc += float(acc.data) * len(t)

print('test accuracy: {:.4f}'.format(sum_acc / len(test_set)))

# 매개변수 저장하기
model.save_weights('my_mlp.npz')

epoch: 1, loss: 1.897141
epoch: 2, loss: 1.263978
epoch: 3, loss: 0.909363
test accuracy: 0.8403


In [9]:
model = MLP((1000, 10))

# 매개변수 읽기
if os.path.exists('my_mlp.npz'):
    model.load_weights('my_mlp.npz')


# 테스트셋 점수 확인
test_set = dezero.datasets.MNIST(train=False)
test_loader = DataLoader(test_set, batch_size)
sum_acc = 0
with dezero.no_grad():
    for x, t in test_loader:
        y = model(x)
        acc = F.accuracy(y, t)
        sum_acc += float(acc.data) * len(t)

print('test accuracy: {:.4f}'.format(sum_acc / len(test_set)))

test accuracy: 0.8403
